In [1]:
import requests
import pandas as pd

# 1. Maç listesini çek
url_list = (
    "https://www.bilyoner.com/api/v3/mobile/aggregator/"
    "gamelist/all/v1"
)
params = {
    "tabType": 1,
    "bulletinType": 2,
    "liveEventsEnabledForPreBulletin": "true"
}
headers = {
    "User-Agent": "Mozilla/5.0"
}

resp = requests.get(url_list, params=params, headers=headers)
resp.raise_for_status()
data = resp.json()

In [2]:
event_items = [(k, v) for k, v in data.items()]
event_items = event_items[0][1]

In [3]:
len(event_items)

108

In [4]:
import random

rows = []

"""
rows.append({
        "gün":     evt.get("strd"),
        "saat":    evt.get("strt"),
        "turnuva": evt.get("slgn"),
        "ev":      evt.get("htn"),
        "dep":     evt.get("atn"),
        "maç_id":  evt.get("id")
    })
"""
# event_items bir sözlük: {eventId: {...}, ...}
event_list = list(event_items.values())  # Dict'ten listeye çevir
random.shuffle(event_list)  # Listeyi bir kere karıştır

smart_analysis_results = {}
for match in event_list:
    print(match["en"])
    gün = match.get("strd")
    saat = match.get("strt")
    turnuva = match.get("slgn")
    ev = match.get("htn")
    dep = match.get("atn")
    maç_id = match.get("id")
    # print(f"{gün} {saat} {turnuva} {ev} {dep} {maç_id}")
    
    url = f"https://www.bilyoner.com/api/mobile/match-card/v2/{maç_id}/smart-analysis"
    resp = requests.get(url, headers=headers)
    
    if resp.ok:
        smart_analysis_results[maç_id] = resp.json()
        # print(f"✔️ {maç_id} için başarıyla alındı.")
    else:
        smart_analysis_results[maç_id] = None
        # print(f"❌ {maç_id} için hata: {resp.status_code}")
    
    print("-" * 50)

Dordrecht - Willem II
--------------------------------------------------
Boden City FC - Luleaa
--------------------------------------------------
Damac FC - AL Fateh SC
--------------------------------------------------
FC Boston - Vermont Green FC
--------------------------------------------------
Atletico Mineiro - Maringa FC
--------------------------------------------------
Crb Al - Santos SP
--------------------------------------------------
Varbergs - Landskrona
--------------------------------------------------
Metz - Reims
--------------------------------------------------
Vardar  - Struga Trim
--------------------------------------------------
Flamengo - Botafogo PB
--------------------------------------------------
Df. Justicia - New Old Boys
--------------------------------------------------
CA Tembetary - Guarani Asu
--------------------------------------------------
Club Bolivar - Blooming
--------------------------------------------------
Ternana Calcio - AS Giana Ermini

KeyboardInterrupt: 

In [ ]:
smart_analysis_results

In [ ]:
# Filtreleme:
filtered_results = {}

for event_id, analysis in smart_analysis_results.items():
    markets = analysis.get("markets", [])

    # markets içindeki her bir market dict'inde 'comment' varsa bu maçı al
    try:
        has_comment = any("comment" in m for m in markets)
    except:
        continue
    if has_comment:
        filtered_results[event_id] = analysis
    # comment içermeyenler otomatik olarak atlanıyor

# Metin dosyasını yazma modunda aç
with open("yorumlu_maclar.txt", "w", encoding="utf-8") as f:
    # f.write(f"Yorum içeren maç sayısı: {len(filtered_results)}\n")
    print(f"Yorum içeren maç sayısı: {len(filtered_results)}")
    
    for eid, data in filtered_results.items():
        # f.write(f"\n=== Maç ID {eid} ===\n")
        print(f"\n=== Maç ID {eid} ===")
        for m in data["markets"]:
            if "comment" in m:
                line = f"- {m['marketName']}: {m['comment']}\n"
                line_ = f"{m['marketName']}: {m['comment']}\n"
                f.write(line_)
                print(line, end="")  # end="" çünkü zaten satır sonunda \n var


In [ ]:
# smart_analysis_results: daha önce API'den çektiğiniz JSON dict

all_odds = []

for event_id, analysis in filtered_results.items():
    for m in analysis.get("markets", []):
        if "comment" in m:
            for outcome in m.get("markets", []):
                all_odds.append({
                    "event_id": event_id,
                    "marketName": m.get("marketName"),
                    "label": outcome.get("label"),
                    "fixedOdds": outcome.get("fixedOddsWeb"),
                    "currentOdds": outcome.get("currentOddsWeb")
                })


from collections import defaultdict
# 1. event_id + marketName bazlı gruplandır
groups = defaultdict(list)
for odd in all_odds:
    key = (odd['event_id'], odd['marketName'])
    groups[key].append(odd)

# 2. Her grupta currentOdds en düşük olana düşür
selected = []
for key, items in groups.items():
    # min currentOdds
    best = min(items, key=lambda x: x['currentOdds'])
    selected.append(best)

from random import shuffle
# 3. Seçilenleri karıştır
shuffle(selected)

# 3. Sonucu yazdır
for s in selected:
    print(s)


In [ ]:
event_items

In [ ]:
rows = []
for odd in selected:
    eid = odd["event_id"]
    match = event_items[str(eid)]

    if not match:
        continue
    rows.append({
        "gün":       match.get("strd"),
        "saat":      match.get("strt"),
        "turnuva":   match.get("slgn"),
        "ev":        match.get("htn"),
        "dep":       match.get("atn"),
        "maç_id":    eid,
        "marketName":odd["marketName"],
        "label":     odd["label"],
        "fixedOdds": odd["fixedOdds"],
        "currentOdds": odd["currentOdds"]
    })

In [ ]:
len(rows)

In [ ]:
# 3. Satırları hazırla
# rows = []
# for odd in selected:

#     if int(odd["event_id"]) == smart_analysis_results[int(odd["event_id"])] or True:
#         match = smart_analysis_results[int(odd["event_id"])]

#         rows.append({
#             "gün": match["strd"],
#             "saat": match["strt"],
#             "turnuva": match["slgn"],
#             "ev": match["htn"],
#             "dep": match["atn"],
#             "maç_id": match["id"],
#             "marketName": odd["marketName"],
#             "label": odd["label"],
#             "fixedOdds": odd["fixedOdds"],
#             "currentOdds": odd["currentOdds"]
#         })

# 4. DataFrame oluştur
df = pd.DataFrame(rows, columns=[
    "gün", "saat", "turnuva", "ev", "dep", "maç_id",
    "marketName", "label", "fixedOdds", "currentOdds"
])

# 5. CSV ve Excel olarak kaydet
df.to_csv("combined_matches.csv", index=False, encoding="utf-8-sig")
df.to_excel("combined_matches.xlsx", index=False)

print("CSV ve Excel dosyaları oluşturuldu:")
print("- combined_matches.csv")
print("- combined_matches.xlsx")
print("\nOluşan tablo:")
print(df)